# Basic example

Multiplication of two integer numbers is pretty simple.
Additional information can be got from Python [documentation](https://docs.python.org/3/library/ctypes.html).

## Write C code

We want to multiply two integers and get integer as the result
```c
int mul(int a, int b) {
    return a * b;
}
```

Notice, that it's not mandatory to have `main` entry point function

You should specify that function is written in `C` and needs to be exported to compile DLL in Visual Studio and use it with `ctypes`

```c
#ifdef __cplusplus
extern "C" {
#endif
__declspec(dllexport) int mul(int a, int b) {

    return a * b;

}
#ifdef __cplusplus
}
#endif
```


## Compile shared library (dll in Windows)

```bash
gcc -shared basics.c -o lib_basics.so
```

## Load created library in Python code

In [1]:
from ctypes import cdll

### In Linux

In [2]:
basics = cdll.LoadLibrary('./lib_basics.so')

### In Windows
```python
basics = cdll.LoadLibrary('lib_basics.dll')
```

## Usage

Now it's pretty simple to call `mul` function just like it was a property of module `basics`

In [3]:
basics.mul(2, 5)

10

See? It's easy

# Dot product of two arrays

Array is a block of memory splitted into chunks of a single type and it's easy to use C arrays with `ctypes`

## C function

Just iterate through each element of arrays, multiply corresponding ones and summarize the result
```c
#include <stdlib.h>

int dot(int *a, int *b, size_t length) {
    int result = 0;
    while (length --> 0) {
        result += a[length] * b[length];
    }
    return result;
}
```

## Create arrays

We should import `int` data type from `ctypes`

In [4]:
from ctypes import c_int

Say, it's needed to multiply 3-dimensional vectors

In [5]:
first = (c_int * 3)(1, 2, 3)

We can create an alias for this data type and use it

In [6]:
vector3D = c_int * 3
second = vector3D(4, 5, 6)

## Call the function

In [7]:
c_result = basics.dot(first, second, 3)
python_result = sum(a * b for a, b in zip([1, 2, 3], [4, 5, 6]))
print('C returned', c_result, 'and Python returned', python_result)

C returned 32 and Python returned 32


In [8]:
basics.dot((c_int*1)(2), (c_int*1)(*[3]), 1)

6

Following examples will cause errors

In [9]:
try:
    vector3D([1, 2, 3])
except:
    print('You cannot pass lists')
try:
    vector3D(0, 1, 2, 3)
except:
    print('Forbidden to provide more elements than it should accept')   

You cannot pass lists
Forbidden to provide more elements than it should accept


# Available types

Following types can be used to pass arguments to `C` functions

| ctypes       | type                                    | C type  Python type |
| -----------  | --------------------------------------  | ------------------- |
| c_bool       | _Bool                                   | bool (1) |
| c_char       | char                                    | 1-character bytes object |
| c_wchar      | wchar_t                                 | 1-character string |
| c_byte       | char                                    | int |
| c_ubyte      | unsigned                                | char   int |
| c_short      | short                                   | int |
| c_ushort     | unsigned                                | short  int |
| c_int        | int                                     | int |
| c_uint       | unsigned                                | int    int |
| c_long       | long                                    | int |

| ctypes       | type                                    | C type  Python type |
| -----------  | --------------------------------------  | ------------------- |
| c_ulong      | unsigned long                           | int |
| c_longlong   | __int64 or long long                    | int |
| c_ulonglong  | unsigned __int64 or unsigned long long  | int |
| c_size_t     | size_t                                  | int |
| c_ssize_t    | ssize_t or Py_ssize_t                   | int |
| c_float      | float                                   | float |
| c_double     | double                                  | float |
| c_longdouble | long double                             | float |
| c_char_p     | char * (NUL terminated)                 | bytes object or None |
| c_wchar_p    | wchar_t * (NUL terminated)              | string or None |
| c_void_p     | void *                                  | int or None |

# Change the long passed to your function

Long number in Python represented as an array of longs:
```
{d0, d1, d2, ...}
```

We can change each one of them, so why not?

Read the [documentation](https://docs.python.org/3/c-api/index.html) about `Python` API for `C`.

## Prepare shared library

C function:
```c
#include <Python.h>

int set_long(PyLongObject* o, long new_value,

             size_t digit) {
             
    o->ob_digit[digit] = new_value;
    
    return 0;
}
```

File should be compiled to shared library (dll in Windows).

Makefile for Linux:
```
FLAGS=-shared
LIBRARIES=-I/usr/include/python3.4
BUILD_LIBRARY=gcc $(FLAGS) $(LIBRARIES)
all:
    $(BUILD_LIBRARY) setters.c -o lib_setters.so
```

## Use shared library

It's handy to create Python wrapper for this C function

In [10]:
from ctypes import cdll, c_long, c_size_t, c_voidp

setters = cdll.LoadLibrary('./lib_setters.so')

def change_long(a, b=0, digit=0):
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Don't forget, that Python interpreter will not create new objects for little integers like `0`, so we should avoid assigning of new values to such numbers, because they will be changed in all places where they're used

In [11]:
from ctypes import c_long, c_size_t, c_voidp

def change_long(a, b=0, digit=0):
    args = (a, b, digit)
    if not all(type(a) is int for a in args):
        raise TypeError('All parameters should be of type "int", '
                        'but {} provided'.format(map(type, args)))
    if a + 0 is a:
        raise ValueError('No way. You don\'t want to break '
                         'your interpreter, right?')
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Let's recall, that we cannot change the value of integers inside Python functions

In [12]:
def variable_info(text, variable):
    print('{:^30}: {:#05x} ({:#x})'.format(text, variable, id(variable)))

def foo(a, new_value):
    a = new_value

a = 2**10
variable_info('Before function call', a)
foo(a, 5)
variable_info('After function call', a)

     Before function call     : 0x400 (0x7fac6ccd0f70)
     After function call      : 0x400 (0x7fac6ccd0f70)


Now forget it and take a look on what we've done

In [13]:
a = 2**10
b = a
variable_info('Before function call', a)
change_long(a, 2, 0)
variable_info('After function call', a)
variable_info('What\'s about b? Here it is', b)

     Before function call     : 0x400 (0x7fac6ccd02d0)
     After function call      : 0x002 (0x7fac6ccd02d0)
  What's about b? Here it is  : 0x002 (0x7fac6ccd02d0)


![hahaha](./lied.jpg)

# Cross product

| i | j | k |
|---|---|---|
|ux |uy |uz |
|vx |vy |vz |

![Cross product](./cross.png)

In [14]:
from numpy import array, cross

basis = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
]

## Plain Python

In [15]:
def py_cross(u, v):
    return [
        u[1] * v[2] - u[2] * v[1],
        u[2] * v[0] - u[0] * v[2],
        u[0] * v[1] - u[1] * v[0]
    ]

py_cross(basis[0], basis[1])

[0, 0, 1]

## NumPy

In [16]:
cross(array(basis[0]), array(basis[1]))

array([0, 0, 1])

## C

It's better not to create new array in `C`, but to provide resulting one to store result in it
```c
int cross(float* u, float* v, float* w) {
    w[0] = u[1] * v[2] - u[2] * v[1];
    w[1] = u[2] * v[0] - u[0] * v[2];
    w[2] = u[0] * v[1] - u[1] * v[0];
    return 0;
}

```

In [17]:
from ctypes import cdll
from numpy import empty_like

c_cross = cdll.LoadLibrary('./lib_cross.so')
u = array(basis[0]).astype('f')
v = array(basis[1]).astype('f')
w = empty_like(u)

def cross_wrapper(u, v, w):
    return c_cross.cross(u.ctypes.get_as_parameter(),
                         v.ctypes.get_as_parameter(),
                         w.ctypes.get_as_parameter())

cross_wrapper(u, v, w)
print(w)

[ 0.  0.  1.]


# Let's run performance tests

In [18]:
from numpy.random import rand

BIG_ENOUGH_INTEGER = int(1E5)

vectors_u = rand(BIG_ENOUGH_INTEGER, 3).astype('f')
vectors_v = rand(BIG_ENOUGH_INTEGER, 3).astype('f')

print('Vectors u:', vectors_u)

Vectors u: [[ 0.78028202  0.93556505  0.68736547]
 [ 0.55792838  0.51500559  0.7040332 ]
 [ 0.90751541  0.958561    0.9576624 ]
 ..., 
 [ 0.54255736  0.23605387  0.73498017]
 [ 0.15223685  0.49756163  0.40691996]
 [ 0.81158608  0.64280111  0.90144485]]


In [19]:
%%timeit
for i in range(BIG_ENOUGH_INTEGER):
    py_cross(vectors_u[i], vectors_v[i])

1 loop, best of 3: 412 ms per loop


In [20]:
%%timeit
cross(vectors_u, vectors_v)

100 loops, best of 3: 2.51 ms per loop


In [21]:
%%timeit
vectors_w = empty_like(vectors_u)

for i in range(BIG_ENOUGH_INTEGER):
    cross_wrapper(vectors_u[i], vectors_v[i], vectors_w[i])

1 loop, best of 3: 3.31 s per loop


## Are calculations right?

In [22]:
from numpy import allclose

np_result = cross(vectors_u, vectors_v)

py_result = [py_cross(vectors_u[i], vectors_v[i])
             for i in range(BIG_ENOUGH_INTEGER)]
print(allclose(np_result, py_result))

vectors_w = empty_like(vectors_u)
assert sum([cross_wrapper(vectors_u[i], vectors_v[i], vectors_w[i])
            for i in range(BIG_ENOUGH_INTEGER)]) == 0
print(allclose(np_result, vectors_w))


True
True


## NumPy versus human: final battle

What we've done wrong? `C` code should be faster! Maybe `Python` loop is an issue?
```c
int cross_vectors(float *u, float *v, float *w,
                  size_t amount) {

    while(amount --> 0) {
        cross(&u[amount * 3], &v[amount * 3],
              &w[amount * 3]);
    }
}
```

It's better to compile with optimization. Also to use `-fPIC` flag to avoid following compilation error
```
relocation against symbol `cross' can not be used when making a shared object
```
What we get:
```
gcc -shared -fPIC cross.c -O3 -o lib_cross.so
```

Numpy arrays are flattened when got as `C` arrays.
Also `len` operator returns amount of rows of matrix. If you want to get summary amount of elements, you should use `size` property.

In [23]:
vectors_w = empty_like(vectors_u)

c_vectors_u = vectors_u.ctypes.get_as_parameter()
c_vectors_v = vectors_v.ctypes.get_as_parameter()
c_vectors_w = vectors_w.ctypes.get_as_parameter()

In [24]:
%%timeit
vectors_w = empty_like(vectors_u)

c_vectors_w = vectors_w.ctypes.get_as_parameter()
c_cross.cross_vectors(c_vectors_u, c_vectors_v, c_vectors_w, len(vectors_u))

1000 loops, best of 3: 646 µs per loop


In [25]:
c_cross.cross_vectors(c_vectors_u, c_vectors_v, c_vectors_w, len(vectors_u))
print(allclose(np_result, vectors_w))

True


Are you surprised?